In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')

In [4]:
def data_processing(data):
    data = data.drop(['id'],axis=1)
    data['BMI'] = data['Weight'] / data['Height']**2
    numerical_columns = data.select_dtypes(include=['float64', 'int64'])
    object_columns = data.select_dtypes(include=['object'])
    
    binary_cols = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
    for col in binary_cols:
        data[col] = data[col].map({'yes': 1, 'no': 0})

    data['Gender'] = data['Gender'].map({'Male':1,'Female':0})
    data['CAEC'] = data['CAEC'].map({
        'no': 0,
        'Always': 3,
        'Frequently': 2,
        'Sometimes': 1
    })

    data['CALC'] = data['CALC'].map({
        'no': 0,
        'Frequently': 2,
        'Sometimes': 1
    })
    
    transport_mapping = {
        'Bike': 2,
        'Walking': 2,
        'Public_Transportation': 1,
        'Automobile': 0,
        'Motorbike': 0,
    }
    data['MTRANS'] = data['MTRANS'].map(transport_mapping)
    
    power_transformer = PowerTransformer(method='box-cox')
    data[numerical_columns.columns] = power_transformer.fit_transform(data[numerical_columns.columns]+1)
    scaler = StandardScaler()
    data[numerical_columns.columns] = scaler.fit_transform(data[numerical_columns.columns])
    return data

In [5]:
def train_processing(data):
    
    
    target_mapping = {
        'Insufficient_Weight': 0,
        'Normal_Weight': 1,
        'Overweight_Level_I': 2,
        'Overweight_Level_II': 3,
        'Obesity_Type_I': 4,
        'Obesity_Type_II': 5,
        'Obesity_Type_III': 6
    }

    data = data.map(target_mapping)
    return data

In [6]:
X = data_processing(train).drop(['NObeyesdad'], axis=1)
X_test = data_processing(test)
t = data_processing(train).NObeyesdad
t = train_processing(t)

In [7]:
from optuna.integration import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop, Adam

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
def train(train_loader):
  model.train()
  running_loss = 0
  for batch_idx, (features, labels) in enumerate(train_loader):
    features = features.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(features)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    loss.backward()
    optimizer.step()
  train_loss = running_loss / len(train_loader)
  return train_loss

def valid(val_loader):
  model.eval()
  running_loss = 0
  correct = 0
  total = 0
  with torch.no_grad():
    for batch_idx, (features, labels) in enumerate(val_loader):
      features = features.to(device)
      labels = labels.to(device)
      outputs = model(features)
      loss = criterion(outputs, labels)
      running_loss += loss.item()
      predicted = outputs.max(1, keepdim=True)[1]
      correct += predicted.eq(labels.view_as(predicted)).sum().item() # ここで精度を計算
      total += labels.size(0)
  val_loss = running_loss / len(val_loader)
  val_acc = correct / total
  return val_loss, val_acc

In [10]:
class MultiClassNet(nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.lin1 = nn.Linear(num_features, 512)
#         self.lin2 = nn.Linear(512, 512)
        self.lin3 = nn.Linear(512, 256)
        self.lin4 = nn.Linear(256, 128)
        self.lin6 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
    
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(128)
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
#         x = self.lin2(x)
#         x = self.bn2(x)
#         x = self.relu(x)
#         x = self.dropout(x)
        
        x = self.lin3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.lin4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.lin6(x)
        return x       

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [12]:
oof_li = []
for fold, (train_index, valid_index) in enumerate(kf.split(X,t)):
    X_train = X.iloc[train_index]
    X_valid = X.iloc[valid_index]
    t_train = t[train_index]
    t_valid = t[valid_index]
    
    
    # LGBM
    lgb_train = lgb.Dataset(X_train, t_train)
    lgb_valid = lgb.Dataset(X_valid, t_valid)
    
    lgb_param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        'verbosity':-1,
        "boosting_type": "gbdt",
        'random_state':0,
        "num_class": 7,
        'device': 'gpu',
    }
    
    gbm = lgb.train(params=lgb_param,
                   train_set=lgb_train,
                   valid_sets=lgb_valid,
                   callbacks=[lgb.early_stopping(stopping_rounds=20,verbose=True), # early_stopping用コールバック関数
#                            lgb.log_evaluation(verbose_eval)
                             ], # コマンドライン出力用コールバック関数
#                    verbose_eval=-1,
                   )
    oof_li.append(gbm.predict(X_test))
    print(f'finish{fold+1}/5 (LGBM)')
    
    # NN
    # 型変換
    X_train = torch.FloatTensor(np.array(X_train))
    y_train = torch.LongTensor(np.array(t_train))
    X_val = torch.FloatTensor(np.array(X_valid))
    y_val = torch.LongTensor(np.array(t_valid))
    
    
    # データセット
    train_datasets = TensorDataset(X_train, y_train)
    val_datasets = TensorDataset(X_val, y_val)
    
    # データローダー
    train_loader = DataLoader(train_datasets, batch_size=3000, shuffle=True)
    val_loader = DataLoader(val_datasets, batch_size=1000, shuffle=False)
    
    num_epochs = 200
    NUM_FEATURES = X_train.shape[1]
    N_CLASSES = len(y_train.unique())
    N_CLASSES, NUM_FEATURES
    
    model = MultiClassNet(NUM_FEATURES, N_CLASSES)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.0001)
    
    loss_list = []
    val_loss_list = []
    val_acc_list = []
    for epoch in range(num_epochs):
      loss = train(train_loader)
      val_loss, val_acc = valid(val_loader)
      if epoch % 50 == 0:
          print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f' % (epoch, loss, val_loss, val_acc))
      # logging
      loss_list.append(loss)
      val_loss_list.append(val_loss)
      val_acc_list.append(val_acc)
    print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f' % (epoch, loss, val_loss, val_acc))
    predicted = model(torch.FloatTensor(np.array(X_test)).to(device)).detach().cpu().numpy()
    oof_li.append(predicted)
    print(f'finish{fold+1}/5 (NN)')
    
    
#     break

[I 2024-02-28 14:35:32,000] A new study created in memory with name: no-name-070541f0-7723-4349-8609-9de718d11f32
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 20 rounds


feature_fraction, val_score: 0.271899:  14%|#4        | 1/7 [00:08<00:49,  8.18s/it]

Early stopping, best iteration is:
[73]	valid_0's multi_logloss: 0.271899
Training until validation scores don't improve for 20 rounds


feature_fraction, val_score: 0.271899:  29%|##8       | 2/7 [00:10<00:23,  4.79s/it]

...

min_child_samples, val_score: 0.258992:  80%|########  | 4/5 [00:20<00:05,  5.23s/it]

Early stopping, best iteration is:
[131]	valid_0's multi_logloss: 0.259965
Training until validation scores don't improve for 20 rounds


min_child_samples, val_score: 0.258992: 100%|##########| 5/5 [00:25<00:00,  5.18s/it]

Early stopping, best iteration is:
[128]	valid_0's multi_logloss: 0.260568


finish5/5 (LGBM)
epoch 0, loss: 1.9110 val_loss: 1.8931 val_acc: 0.2980
epoch 50, loss: 0.4839 val_loss: 0.4165 val_acc: 0.8663
epoch 100, loss: 0.3896 val_loss: 0.3459 val_acc: 0.8829
epoch 150, loss: 0.3598 val_loss: 0.3288 val_acc: 0.8827
epoch 199, loss: 0.3313 val_loss: 0.3218 val_acc: 0.8865
finish5/5 (NN)


In [13]:
oof_array = np.array(oof_li)
mean_oof = np.mean(oof_array, axis=0)
# oof_array.shape, mean_oof.shape

In [14]:
ensem_target = np.argmax(mean_oof,axis=1)
ensem_target

array([5, 2, 6, ..., 0, 1, 5])

In [15]:
def test_processing(data): 
    target_mapping = {
        0:'Insufficient_Weight',
        1:'Normal_Weight',
        2:'Overweight_Level_I',
        3:'Overweight_Level_II',
        4:'Obesity_Type_I',
        5:'Obesity_Type_II',
        6:'Obesity_Type_III'
    }

    # 辞書を用いて変換
    data_mapped = np.vectorize(target_mapping.get)(data)

    return data_mapped

In [16]:
target = test_processing(ensem_target)
target

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype='<U19')

In [17]:
submit = sample

In [18]:
submit['NObeyesdad'] = target
submit

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [20]:
submit.to_csv('/kaggle/working/submission.csv',index=False)